In [15]:
from operator import attrgetter
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IGNORE_INDEX
from llava.conversation import conv_templates, SeparatorStyle

import torch
import cv2
import numpy as np
from PIL import Image
import requests
import copy
import warnings
from decord import VideoReader, cpu

import pandas as pd
import json

from llama_cpp import Llama

In [22]:
from llama_cpp import Llama

model_kwargs = {
    "n_ctx": 4096,    # Context length to use
    "n_threads": 4,   # Number of CPU threads to use
    "n_gpu_layers": 1, # Number of model layers to offload to GPU
}

llm = Llama(model_path="Phi-3-mini-4k-instruct-q4.gguf", **model_kwargs)


llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count_kv u32           

In [29]:
with open('top_100_songs.json', 'r') as file:
        songs = [json.loads(line) for line in file]

# Prepare the input for the LLM
input_text = f"Video description: A happy and energetic scene with lots of movement and excitement.\n\nSongs Attribute:\n"
for index, song in enumerate (songs):
    input_text += f"Song Title {index}: {song['Song']}, Mood: {song['Mood']}, Genre: {song['Genre']}, Description: {song['Music Description']}\n"

input_text += "\nSelect the top 5 appropriate background songs for the given video description in list form. Give only the title songs in list format"
print(input_text)

Video description: A happy and energetic scene with lots of movement and excitement.

Songs Attribute:
Song Title 0: SUNSET DREAMS, Mood: Relaxing, Genre: Jazz, Description: A calm journey about enjoying life’s quiet moments
Song Title 1: STARRY NIGHTS, Mood: Serene, Genre: Classical, Description: A song about stargazing and finding peace
Song Title 2: HORIZON BOUND, Mood: Uplifting, Genre: Pop, Description: A story of chasing dreams and new beginnings
Song Title 3: OCEAN WAVES, Mood: Nostalgic, Genre: Blues, Description: Reflecting on memories by the seaside
Song Title 4: BURNING SOUL, Mood: Intense, Genre: Metal, Description: Expressing inner struggles and resilience
Song Title 5: COUNTRY ROADS, Mood: Heartwarming, Genre: Country, Description: A song about traveling through rural landscapes
Song Title 6: CITY LIGHTS, Mood: Energetic, Genre: Disco, Description: Experiencing nightlife and the energy of the city
Song Title 7: FALLING RAIN, Mood: Reflective, Genre: Jazz, Description: Abo

In [30]:
output = llm(
      input_text, # Prompt
      max_tokens=1000, # Generate up to 32 tokens
      stop=["Q:"], # Stop generating just before the model would generate a new question
      echo=False # Do not Echo the prompt back in the output
) # Generate a completion, can also call create_completion

print(output['choices'][0]['text'])


ValueError: Requested tokens (3965) exceed context window of 512

In [21]:
print(response)

<generator object Llama.generate at 0x00000219AB36B240>


In [7]:
def excel_to_json(excel_path, json_path):
    df = pd.read_excel(excel_path)
    df.to_json(json_path, orient='records', lines=True)

In [10]:
import json
import torch

def give_top(json_path, video_description):
    model.eval()

    with open(json_path, 'r') as file:
        songs = [json.loads(line) for line in file]

    # Prepare the input for the LLM
    input_text = f"Video description: {video_description}\n\n csv description of 100 songs:\n"
    for index, song in enumerate (songs):
        input_text += f"Song Title {index}: {song['Song']}, Mood: {song['Mood']}, Genre: {song['Genre']}, Description: {song['Music Description']}\n"

    input_text += "\nSelect the most appropriate 5 background songs for the given video description using the provided csv, in list form. For instance:\n- 1) CRYSTAL SKY\n- 2)DANCE FLOOR MAGIC"
    print(input_text)

    # Prepare conversation input
    conv_template = "qwen_1_5"

    conv = copy.deepcopy(conv_templates[conv_template])
    conv.append_message(conv.roles[0], input_text)
    conv.append_message(conv.roles[1], None)
    prompt_question = conv.get_prompt()
    input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
    cont = model.generate(
        input_ids,
        do_sample=False,
        temperature=0.4,
        max_new_tokens=4096,
        modalities=["text"],
    )

    description = tokenizer.batch_decode(cont, skip_special_tokens=True)
    return description

In [ ]:
# Convert Excel to JSON
excel_to_json('Copyright Songs.xlsx', 'top_100_songs.json')

# Get top 5 songs for a given video description
video_description = "A happy and energetic scene with lots of movement and excitement."
top_songs = give_top('top_100_songs.json', video_description)
print("top_songs:",  top_songs)

Video description: A video of a funeral

 csv description of 100 songs:
Song Title 0: SUNSET DREAMS, Mood: Relaxing, Genre: Jazz, Description: A calm journey about enjoying life’s quiet moments
Song Title 1: STARRY NIGHTS, Mood: Serene, Genre: Classical, Description: A song about stargazing and finding peace
Song Title 2: HORIZON BOUND, Mood: Uplifting, Genre: Pop, Description: A story of chasing dreams and new beginnings
Song Title 3: OCEAN WAVES, Mood: Nostalgic, Genre: Blues, Description: Reflecting on memories by the seaside
Song Title 4: BURNING SOUL, Mood: Intense, Genre: Metal, Description: Expressing inner struggles and resilience
Song Title 5: COUNTRY ROADS, Mood: Heartwarming, Genre: Country, Description: A song about traveling through rural landscapes
Song Title 6: CITY LIGHTS, Mood: Energetic, Genre: Disco, Description: Experiencing nightlife and the energy of the city
Song Title 7: FALLING RAIN, Mood: Reflective, Genre: Jazz, Description: About solitude and listening to ra